In [16]:
import pandas as pd
import os
import fnmatch
import numpy as np
import json
from datetime import datetime
basePath='//mxns.loc//shares//NA-Instruments//VIDAS//VIDAS_LIMS//'
colsOrder=['Sample.Sample_Number', 'Test.Analysis','Result.Name', 'Result.Entry', 'Result.Entered_By','Result.Instrument']

In [17]:
#function to generate the result name
def generateResultname(testIdentifier,variable):
    if(variable=='R2'):
        return str(testIdentifier)
    else:
        return str('OD'+str(testIdentifier))

In [ ]:
#fucntion to Create a folder if it dosen't exsist.
def createFolderIfNotExsist(folderPath):
    if os.path.exists(folderPath):
        pass
    else:
        os.makedirs(folderPath)

In [ ]:
#Moce to ARK folder
def moveFiletoArkFolder(arkPath,sourcePath,fname):
    arkDest = arkPath + fname[:-4] + '.bak' #Backup the processed file
    os.rename(sourcePath, arkDest)
    time.sleep(1)

In [ ]:
#write the file
def writeFile(dframe,lab,instName,arkPath,basePath,sourcePath):
    timestr = time.strftime("%Y%m%d%H%M%S")
    importerFilePath = basePath + lab+'_IMPORTER//'+instName+'_' + timestr + '.csv'
    dframe.to_csv(importerFilePath, index=False, encoding="utf-8",quotechar='"',quoting=csv.QUOTE_NONNUMERIC) # write  out the file to be imported
    time.sleep(1)

In [18]:
with open('.\configFiles\VIDASParserConfig.json', 'r') as f:
    config = json.loads(f)
    for d1 in config:
        lab=d1
        labcount=int(config[d1]['count'])
        labImporterPath=config[d1]['importerPath']

In [19]:
createFolderIfNotExsist(labImporterPath)
li = []
print('Processing Files for :'+lab)
for each in range(1,labcount+1):
    instName=lab+'VIDAS'+str(each)
    sourceFolderPath=basePath+instName+'//'
    arkFolderPath=sourceFolderPath+'Archive//'+datetime.now().strftime('%Y')+'//'+datetime.now().strftime('%m_%d')+'//'
    #checking the exsitnace of importer path
    createFolderIfNotExsist(sourceFolderPath)
    #checking the exsitnace of Archive directory and path
    createFolderIfNotExsist(arkFolderPath)
    files = os.listdir(sourceFolderPath)
    for filename in files:
        if fnmatch.fnmatch(filename,'*.CSV'):
            filePath = sourceFolderPath + str(filename)
            try:
                rawdf = pd.read_csv(filePath,index_col=None,header=None)
                rawdf['instrumentName']=instName
                li.append(rawdf)
                print('Processed :'+filename)
                moveFiletoArkFolder(arkFolderPath,sourceFolderPath,filename)
            except:
                continue
raw_data = pd.concat(li, axis=0, ignore_index=True)
raw_data['specimenIdentifier'] =raw_data[0]
raw_data['testIdentifier'] =raw_data[1]
raw_data['R1'] =raw_data[5]
raw_data['R2'] =raw_data[2]
raw_data['patientIdentifier'] = raw_data[4]
raw_data = raw_data[np.isfinite(raw_data["patientIdentifier"])]
raw_data['patientIdentifier']=raw_data['patientIdentifier'].astype(int)
raw_data["samplenumber"] = raw_data["patientIdentifier"].map(str) + raw_data["specimenIdentifier"].str[-6:]
dfResult = pd.melt(raw_data, 
                   id_vars=['instrumentName','specimenIdentifier','testIdentifier','samplenumber'], value_vars=['R1', 'R2'])
dfResult['Result.Name'] = dfResult.apply(lambda x: generateResultname(x.testIdentifier,x.variable),axis=1)
dfResult = dfResult.sort_values(by=['samplenumber'])
dfResult['Test.Analysis'] = 'VIDAS'
dfResult['Result.Entered_By'] = lab+'_VIDAS_IMPORTER'
dfResult = dfResult.rename(index=str,
columns={'samplenumber': 'Sample.Sample_Number', 'value': 'Result.Entry','instrumentName':'Result.Instrument'})
dfResult=dfResult[colsOrder]
#Writes the File to the  ARK, Importer Folders
writeFile(dfResult,lab,instName,arkFolderPath,basePath,sourceFolderPath)

In [25]:
def processEachLabFiles(lab,labcount,labImporterPath,basePath):